In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        pass

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from random import  shuffle
import os
import tensorflow as tf
from random import shuffle

In [3]:
IMG_SIZE=224
X=[]
Y=[]
training_data=[]
test_data=[]
X_t=[]
Y_t=[]

In [4]:
path="/kaggle/input/dogs-cats-images/dataset/training_set/"
for i in os.listdir(path+'cats/'):
    img=cv2.imread(path+'cats/'+i)
    img=cv2.resize(img,(IMG_SIZE,IMG_SIZE))
    training_data.append(tuple([img,1]))
    #X.append(img)
    #Y.append(1)
for i in os.listdir(path+'dogs/'):
    img=cv2.imread(path+'dogs/'+i)
    img=cv2.resize(img,(IMG_SIZE,IMG_SIZE))
    training_data.append(tuple([img,0]))
    #X.append(img)
    #Y.append(0)

shuffle(training_data)
for features,labels in training_data:
    X.append(features)
    Y.append(labels)

del training_data

path="/kaggle/input/dogs-cats-images/dataset/test_set/"
for i in os.listdir(path+'cats/'):
    img=cv2.imread(path+'cats/'+i)
    img=cv2.resize(img,(IMG_SIZE,IMG_SIZE))
    test_data.append(tuple([img,1]))
    #X.append(img)
    #Y.append(1)
for i in os.listdir(path+'dogs/'):
    img=cv2.imread(path+'dogs/'+i)
    img=cv2.resize(img,(IMG_SIZE,IMG_SIZE))
    test_data.append(tuple([img,0]))
    #X.append(img)
    #Y.append(0)

shuffle(test_data)
for features,labels in test_data:
    X_t.append(features)
    Y_t.append(labels)

del test_data


In [5]:
X=np.array(X)
#X=X/225.0
Y=np.array(Y)
X=np.array(X).reshape(-1,IMG_SIZE,IMG_SIZE,3)
X.shape

X_t=np.array(X_t)
#X_t=X-t/225.0
Y_t=np.array(Y_t)
X_t=np.array(X_t).reshape(-1,IMG_SIZE,IMG_SIZE,3)
X_t.shape

(2000, 224, 224, 3)

In [11]:
class myCallBack(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs={}):
        if(logs.get('accuracy') > 0.95):
            self.model.stop_training = True

callback=myCallBack()
        

In [12]:
model=tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(64,(3,3), input_shape=X.shape[1:],activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2,2))

model.add(tf.keras.layers.Conv2D(64,(3,3),activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2,2))

model.add(tf.keras.layers.Conv2D(64,(3,3),activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2,2))

model.add(tf.keras.layers.Conv2D(64,(3,3),activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2,2))

model.add(tf.keras.layers.Conv2D(64,(3,3),activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2,2))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256,
                                activation='relu',
                                kernel_regularizer=tf.keras.regularizers.l2(0.01)))
#model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(256,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01)))
#model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(256,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(tf.keras.layers.Dense(2,activation='softmax'))

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])



model.fit(X,Y,epochs=30,validation_data=(X_t,Y_t),callbacks=[callback])

Epoch 1/30
250/250 [==============================] - 9s 36ms/step - loss: 5.1580 - accuracy: 0.5351 - val_loss: 3.3609 - val_accuracy: 0.5990
Epoch 2/30
250/250 [==============================] - 9s 34ms/step - loss: 2.7390 - accuracy: 0.6309 - val_loss: 2.2674 - val_accuracy: 0.6105
Epoch 3/30
250/250 [==============================] - 9s 34ms/step - loss: 1.9220 - accuracy: 0.6636 - val_loss: 1.6777 - val_accuracy: 0.6460
Epoch 4/30
250/250 [==============================] - 9s 34ms/step - loss: 1.4285 - accuracy: 0.6975 - val_loss: 1.2495 - val_accuracy: 0.7140
Epoch 5/30
250/250 [==============================] - 9s 35ms/step - loss: 1.0921 - accuracy: 0.7414 - val_loss: 0.9968 - val_accuracy: 0.7435
Epoch 6/30
250/250 [==============================] - 9s 34ms/step - loss: 0.8669 - accuracy: 0.7729 - val_loss: 0.7917 - val_accuracy: 0.7780
Epoch 7/30
250/250 [==============================] - 9s 34ms/step - loss: 0.7197 - accuracy: 0.7930 - val_loss: 0.7121 - val_accuracy: 0.7700

In [13]:
model.save('saved_model.h5')